<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Quantum State Tomography

* **Last Updated:** March 8, 2019
* **Requires:** qiskit-terra 0.7, qiskit-ignis 0.1, qiskit-aer 0.1

This notebook contains examples for using the ``ignis.verification.tomography`` state tomography module.

In [28]:
# Needed for functions
import numpy as np
import time

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

## 2-Qubit state tomography Example

In the below example we want to perform state tomography on a 2Q Bell state between qubits 3 and 5. To make the reference circuit we generate the expected statevector using ``statevector_simulator`` between qubits 0 and 1. 

In [29]:
# Create the expected density matrix
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
print(bell)

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

         ┌───┐     
q8_0: |0>┤ H ├──■──
         └───┘┌─┴─┐
q8_1: |0>─────┤ X ├
              └───┘
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [30]:
# Create the actual circuit 
q2 = QuantumRegister(6)
bell = QuantumCircuit(q2)
bell.h(q2[3])
bell.cx(q2[3], q2[5])
print(bell)

                   
q9_0: |0>──────────
                   
q9_1: |0>──────────
                   
q9_2: |0>──────────
         ┌───┐     
q9_3: |0>┤ H ├──■──
         └───┘  │  
q9_4: |0>───────┼──
              ┌─┴─┐
q9_5: |0>─────┤ X ├
              └───┘


Here we are going to generate and fun the state tomography circuits. By only passing in the 2 registers we want to measure the state tomography will only run on that reduced $2^2$ Hilbert space. However, if we pass the whole register in the state tomography module will try and fit the full $2^6$ space.

In [31]:
# Generate circuits and run on simulator
t = time.time()
# Generate the state tomography circuits. Only pass in the 
# registers we want to measure (in this case 3 and 5)
qst_bell = state_tomography_circuits(bell, [q2[3],q2[5]])
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
print('Time taken:', time.time() - t)

# Generate the state tomography circuits using the default settings for 
# basis

tomo_bell = StateTomographyFitter(job.result(), qst_bell)

Time taken: 0.4744570255279541


The fitter will output a density matrix ordered according to how we passed in the registers to ``state_tomography_circuits``. 

In [32]:
# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity =', F_bell)

Fit Fidelity = 0.9999000849303845


### Repeat the Example with Measurement Noise

In [33]:
#Add measurement noise
noise_model = noise.NoiseModel()
for qi in range(6):
    read_err = noise.errors.readout_error.ReadoutError([[0.75, 0.25],[0.1,0.9]])
    noise_model.add_readout_error(read_err,[qi])
    
#generate the calibration circuits
meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[3,5])

backend = Aer.get_backend('qasm_simulator')
qobj_cal = qiskit.compile(meas_calibs, backend=backend, shots=15000)
qobj_tomo = qiskit.compile(qst_bell, backend=backend, shots=15000)

job_cal = backend.run(qobj_cal, noise_model=noise_model)
job_tomo = backend.run(qobj_tomo, noise_model=noise_model)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

#no correction
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity (no correction) =', F_bell)

#correct data
correct_tomo_results = meas_fitter.filter.apply(job_tomo.result(), method='least_squares')
tomo_bell = StateTomographyFitter(correct_tomo_results, qst_bell)
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity (w/ correction) =', F_bell)

Fit Fidelity (no correction) = 0.5769036647742336
Fit Fidelity (w/ correction) = 0.9967713813311038


## Generating and fitting random states

We now test the functions on state generated by a circuit consiting of a layer of random single qubit unitaries u3

In [34]:
def random_u3_tomo(nq, shots):
    
    def rand_angles():
        return tuple(2 * np.pi * np.random.random(3) - np.pi)
    q = QuantumRegister(nq)
    circ = QuantumCircuit(q)
    for j in range(nq):
        circ.u3(*rand_angles(), q[j])
    job = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
    psi_rand = job.result().get_statevector(circ)
    
    qst_circs = state_tomography_circuits(circ, q)
    job = qiskit.execute(qst_circs, Aer.get_backend('qasm_simulator'),
                         shots=shots)
    tomo_data = StateTomographyFitter(job.result(), qst_circs)
    rho_cvx = tomo_data.fit(method='cvx')
    rho_lstsq = tomo_data.fit(method='lstsq')
    
    print('F fit (CVX) =', state_fidelity(psi_rand, rho_cvx))
    print('F fit (LSTSQ) =', state_fidelity(psi_rand, rho_lstsq))

In [35]:
for j in range(5):
    print('Random single-qubit unitaries: set {}'.format(j))
    random_u3_tomo(3, 5000)

Random single-qubit unitaries: set 0
F fit (CVX) = 0.9982342978400156
F fit (LSTSQ) = 0.9955699556857641
Random single-qubit unitaries: set 1
F fit (CVX) = 0.9997462856276226
F fit (LSTSQ) = 0.9981847872964353
Random single-qubit unitaries: set 2
F fit (CVX) = 0.9968201908496568
F fit (LSTSQ) = 0.9923331569383647
Random single-qubit unitaries: set 3
F fit (CVX) = 0.9993092033884361
F fit (LSTSQ) = 0.995875515175604
Random single-qubit unitaries: set 4
F fit (CVX) = 0.9957937064209796
F fit (LSTSQ) = 0.9932656872439047


## 5-Qubit Bell State

In [36]:
# Create a state preparation circuit
q5 = QuantumRegister(5)
bell5 = QuantumCircuit(q5)
bell5.h(q5[0])
for j in range(4):
    bell5.cx(q5[j], q5[j + 1])

# Get ideal output state
job = qiskit.execute(bell5, Aer.get_backend('statevector_simulator'))
psi_bell5 = job.result().get_statevector(bell5)

# Generate circuits and run on simulator
t = time.time()
qst_bell5 = state_tomography_circuits(bell5, q5)
job = qiskit.execute(qst_bell5, Aer.get_backend('qasm_simulator'), shots=5000)

# Extract tomography data so that counts are indexed by measurement configuration
tomo_bell5 = StateTomographyFitter(job.result(), qst_bell5)
print('Time taken:', time.time() - t)

Time taken: 11.79207706451416


In [37]:
t = time.time()
rho_lstsq_bell5 = tomo_bell5.fit(method='lstsq')
print('Least-Sq Reconstruction')
print('Time taken:', time.time() - t)
print('Fit Fidelity:', state_fidelity(psi_bell5, rho_lstsq_bell5))

Least-Sq Reconstruction
Time taken: 6.593528985977173
Fit Fidelity: 0.9934254969851912


In [38]:
t = time.time()
rho_cvx_bell5 = tomo_bell5.fit(method='cvx')
print('CVX Reconstruction')
print('Time taken:', time.time() - t)
print('Fidelity:', state_fidelity(psi_bell5, rho_cvx_bell5))

CVX Reconstruction
Time taken: 73.93138408660889
Fidelity: 0.9999122713296319
